In [1]:
import pandas as pd

In [2]:
df = pd.read_html("https://en.wikipedia.org/wiki/Number_of_guns_per_capita_by_country")[0]

In [3]:
df.columns = df.iloc[0]
df = df.reindex(df.index.drop(0))

In [4]:
df.head()

,Country,Guns per 100 Residents (2014),Rank (2014),Notes
1,United States,112.6[7],1,"Most guns are regulated at a state level, not ..."
2,Serbia,69.7,2,Not including Kosovo
3,Yemen,54.8,3,NaN
4,Switzerland,45.7,4,"Including the personal weapons of the militia,..."
5,Cyprus,36.1,5,NaN


In [5]:
df.iloc[0,1] = 112.6

In [6]:
import matplotlib.pyplot as plt

In [7]:
df.dtypes

0
Country                          object
Guns per 100 Residents (2014)    object
Rank (2014)                      object
Notes                            object
dtype: object

In [8]:
df = df.convert_objects(convert_numeric=True)

In [9]:
df.head(15)

,Country,Guns per 100 Residents (2014),Rank (2014),Notes
1,United States,112.6,1,"Most guns are regulated at a state level, not ..."
2,Serbia,69.7,2,Not including Kosovo
3,Yemen,54.8,3,NaN
4,Switzerland,45.7,4,"Including the personal weapons of the militia,..."
5,Cyprus,36.1,5,NaN
6,Saudi Arabia,35.0,6,NaN
7,Iraq,34.2,7,NaN
8,Uruguay,31.8,8,NaN
9,Sweden,31.6,9,According to the Swedish National Police Agenc...
10,Norway,31.3,10,NaN


In [10]:
gis = GIS("https://deldev.maps.arcgis.com", "deldev", "")

In [11]:
path = "content/features/analyze"
postdata = {
    "f": "pjson",
    "text" : df.to_csv(),
    "filetype" : "csv",
    "analyzeParameters" : {
        "enableGlobalGeocoding": "true",
        "sourceLocale":"en-us",
        "locationType":"address",
        "sourceCountry":"",
        "sourceCountryHint":""
    }
}


res = gis._portal.con.post(path, postdata)
#import json
#json.dumps(res)
res['publishParameters'].update({"addressFields":{"CountryCode":"Country"}})

path = "content/features/generate"
postdata = {
    "f": "pjson",
    "text" : df.to_csv(),
    "filetype" : "csv",
    "publishParameters" : json.dumps(res['publishParameters'])
}

res2014 = gis._portal.con.post(path, postdata,  use_ordered_dict=True)
#print(json.dumps(res2014))
fc2014 = FeatureCollection(res2014['featureCollection']['layers'][0])

In [12]:
map = MapView()

In [13]:
map

In [14]:
map.add_layer(fc)

In [15]:
item_properties = {
    "title": "Worldwide gun ownership",
    "tags" : "guns,violence",
    "snippet": " GSR Worldwide gun ownership",
    "description": "test description",
    "text": json.dumps({"featureCollection": {"layers": [ fc ]}}),
    "type": "Feature Collection",
    "typeKeywords": "Data, Feature Collection, Singlelayer",
    "extent" : "-102.5272,-41.7886,172.5967,64.984"
}
item = gis.content.create(item_properties)


In [16]:
item

{'access': 'private',
 'accessInformation': None,
 'appCategories': [],
 'avgRating': 0,
 'banner': None,
 'commentsEnabled': True,
 'created': 1449819341000,
 'culture': 'english (united states)',
 'description': 'test description',
 'documentation': None,
 'extent': [[-102.5272, -41.7886], [172.5967, 64.984]],
 'guid': None,
 'id': '21120904f6c74f3cab7b50048fe9c2f6',
 'industries': [],
 'itemControl': 'admin',
 'languages': [],
 'largeThumbnail': None,
 'licenseInfo': None,
 'listed': False,
 'modified': 1449819341000,
 'name': None,
 'numComments': 0,
 'numRatings': 0,
 'numViews': 0,
 'owner': 'deldev',
 'ownerFolder': None,
 'properties': None,
 'protected': False,
 'screenshots': [],
 'size': 45052,
 'snippet': ' GSR Worldwide gun ownership',
 'spatialReference': None,
 'tags': ['guns', 'violence'],
 'thumbnail': None,
 'title': 'Worldwide gun ownership',
 'type': 'Feature Collection',
 'typeKeywords': ['Data', 'Feature Collection', 'Singlelayer'],
 'url': None}